In [22]:
import $file.hw6stdlib
import hw6stdlib._

Compiling hw6stdlib.sc


import $file.$        

import hw6stdlib._

# Homework 6

Due 10/20 at 11:59pm

In this assignment we will write the first full interpreter for Lettuce. Remember that lettuce is a functional language with let bindings. It is very similiar to a language called ML. Here we want to implement the interpreter for the language from the ground up. We will use most of the existing standard library we have developed while we write this, especially the equality functions and Dictionary data type from the last homework.

## Submission Instructions
Upload only this .ipynb file to Canvas. Do not add anything to stdlib since you can't submit it.

In this homework we will define integers and their operations, then build an interpreter.

See [this link](https://www.notion.so/Guidelines-for-Programming-Homework-dbd25efa7bb24915ae6bcb06827fc5b6) for what is and isn't allowed in your code. 



## Problem 1 - The Values (5 points)

Recall that we define interpreters as functions that take in expressions in the form of abstract syntax and give a value as output. In symbols:

$$
eval : Expr \rightarrow Value
$$

It follows that we will need to create a datatype to represent the values that may be computed by lettuce. Bellow is the grammar that defines lettuce values. Implemenet this as a `sealead trait` in Scala below:

$$
\begin{align}
\textbf{Value} ::=&\ NumVal\ \mathbb{Z}\\
\mid&\ BinVal\ \mathbb{B}\\
\mid&\ Error
\end{align}
$$

In [23]:
sealed trait Value
case object Error extends Value
case class NumVal(x: Integer) extends Value
case class BinVal(y: Bool) extends Value

defined trait Value
defined object Error
defined class NumVal
defined class BinVal

If your definition was correct then the values in the cell below should compile.

In [24]:
val v1 = NumVal(Positive(Succ(Succ(Zero))))
val v2 = BinVal(False)
val v3 = Error
passed(5)


*** Tests Passed (5 points) ***


v1: NumVal = NumVal(Positive(Succ(Succ(Zero))))
v2: BinVal = BinVal(False)
v3: Error.type = Error

## Problem 2 - The Expressions (10 points)

There are many possible expressions in the Lettuce language. We also use an abstract syntax to represent this. Here is the grammar to remind you of all the syntactic elements in Lettuce:

$$\begin{array}{rcll}
\mathbf{Expr} & ::= & Const(\mathbb{Z}) \\
 & | & Bin(\mathbb{B}) \\
 & | & Ident(\mathbf{String}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Pow (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Neg (\mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{String}, \mathbf{Expr}, \mathbf{Expr}) \\
\end{array}$$

Now define a `sealed trait` for expressions in Lettuce:

In [25]:
sealed trait Expr
case class Const(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr

defined trait Expr
defined class Const
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let

If you defined `Expr` properly then the following expressions should compile:

In [26]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
passed(5)


*** Tests Passed (5 points) ***


ex1: Const = Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero)))))))
ex2: Const = Const(Negative(Succ(Succ(Succ(Zero)))))
ex3: Bin = Bin(True)
ex4: Ident = Ident("x")

In [27]:
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))
passed(5)


*** Tests Passed (5 points) ***


ex5: Plus = Plus(Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Const(Negative(Succ(Succ(Succ(Zero))))))
ex6: Let = Let(x,Plus(Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Const(Negative(Succ(Succ(Succ(Zero)))))),Mult(Ident(x),Ident(x)))

## Problem 3 - The Interpreter (30 pts)

We now need to define the interpreter for this language. It is a function of the form:

$$
eval : Expr \rightarrow Value
$$

Except now that we have an execution environment($\sigma$) we need some way to include sigma in this function. This will be as an additional argument:

$$
eval : \sigma \rightarrow Expr \rightarrow Value
$$

We will let $\sigma$ be a `Dictionary` as we defined in last week's homework. To add something to a dicitonary use the `Entry` constructor to add on a new element. This should be very similiar to the way `Cons` works on lists.

Below we have given you the skeleton of the function `eval`. Fill in each case for the interpreter so that it can interpret all possible Lettuce expressions. The inference rules we covered in the classroom should be very helpful with this.

Hint: It may be helpful to define some helper functions for the inference rules that have similar forms. These are the rules like binary operations for arithmetic, bool, etc. Just like we lumped some of these rules together into a single rule, we can do the same for our interpreter in the form of auxillary functions. The bonus `eval_bin` function from Homework 4 may be a good place for inspiration.

Hint: Be ready to write some nested case matches. Most of your cases should have two nestings. One for the Sytnax element and another for evaluating the expressions it may contain. This is not true for all expressions but most will 

In [28]:
/*def eval_bin_int(x: Dictionary[String, Value], y: Expr, z: Expr, f:(Integer, Integer)=>Integer): Value = {
    NumVal(f(eval(x, y), eval(x, z)))
}

def eval_bin_bool(x: Dictionary[String, Value], y: Expr, z: Expr, f:(Value, Value)=>Bool): Value ={
    BinVal(f(eval(x, y), eval(x, z)))
}*/

def eval(dict: Dictionary[String, Value], z: Expr): Value = z match{
    case Bin(x) => BinVal(x)
    case Const(x) => NumVal(x)
    case Ident(x) => lookup(string_eq, dict, x) match{
        case None => Error
        case Just(y) => y match{
            case NumVal(y) => NumVal(y)
            case BinVal(y) => BinVal(y)
        }
    }
    case Mult(x, y) => (eval(dict,x), eval(dict, y)) match{
        case(NumVal(x), NumVal(y)) => NumVal(mult(x, y))
    }
    case Plus(x, y) => (eval(dict,x), eval(dict, y)) match{
        case(NumVal(x), NumVal(y)) => NumVal(plus(x, y))
    }
    case Minus(x, y) => (eval(dict, x), eval(dict,y)) match{
        case(NumVal(x), NumVal(y)) => NumVal(minus(x, y))
    }
    case Pow(x, y) => (eval(dict, x), eval(dict,y)) match{
        case(NumVal(x), NumVal(y)) => y match{
            case Positive(y) => NumVal(pow(x, y))
        }
    }
    case Neg(x) => eval(dict, x) match{
        case Error => Error
        case NumVal(x) => x match{
            case x => NumVal(negate(x))
        }
        case BinVal(x) => x match{
            case True => BinVal(False)
            case False => BinVal(True)
        }
    }
    case Let(id, x, y) => eval(dict, x) match {
        case Error => Error
        case x => eval(Entry(id, x, dict), y)
    }
}

defined function eval

If this was defined correctly then the definitions above should evaluate:

In [29]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))

ex1: Const = Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero)))))))
ex2: Const = Const(Negative(Succ(Succ(Succ(Zero)))))
ex3: Bin = Bin(True)
ex4: Ident = Ident("x")
ex5: Plus = Plus(Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Const(Negative(Succ(Succ(Succ(Zero))))))
ex6: Let = Let(x,Plus(Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Const(Negative(Succ(Succ(Succ(Zero)))))),Mult(Ident(x),Ident(x)))

In [30]:
assert(eval(EmptyDict, ex3) == BinVal(True))
passed(10)


*** Tests Passed (10 points) ***


In [31]:
assert(eval(EmptyDict, ex4) == Error)
passed(10)


*** Tests Passed (10 points) ***


In [32]:
assert(eval(EmptyDict, ex5) == NumVal(Positive(two)))
assert(eval(EmptyDict, ex6) == NumVal(Positive(four)))
passed(10)


*** Tests Passed (10 points) ***
